In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=46)

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=46)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=46)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9975944190522011
0.8701923076923077


In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00883412, 0.00788069, 0.0069983 , 0.00816703, 0.00605226]), 'score_time': array([0.00104213, 0.00101638, 0.00221896, 0.00165939, 0.00099254]), 'test_score': array([0.85288462, 0.85865385, 0.88161694, 0.8719923 , 0.88546679])}


In [8]:
import numpy as np
print(np.mean(scores['test_score']))

0.8701228992374326


In [9]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8701228992374326


In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=46)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8689606491774121


In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [12]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=46), params, n_jobs=-1)

In [13]:
gs.fit(train_input, train_target)
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9657494708485664


In [14]:
print(gs.best_estimator_)

DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=46)


In [15]:
print(gs.cv_results_['mean_test_score'])

[0.87781909 0.87473717 0.87223532 0.86877027 0.86857814]


In [16]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [17]:
params = {'min_impurity_decrease':np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

In [18]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=46), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)

{'max_depth': 15, 'min_impurity_decrease': 0.0001, 'min_samples_split': 2}


In [19]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8774341082401718


In [20]:
from scipy.stats import uniform, randint

In [22]:
rgen = randint(0, 10)
rgen.rvs(10)

array([0, 5, 9, 0, 3, 8, 6, 5, 9, 0], dtype=int64)

In [23]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64),
 array([102, 102,  98,  90,  90, 117,  79, 116,  97, 109], dtype=int64))

In [24]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.78477756, 0.12653941, 0.94504428, 0.46754819, 0.38335316,
       0.50675166, 0.00987057, 0.59555596, 0.11510812, 0.85916753])

In [25]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25),
          }

In [26]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=46), params,
                        n_iter=100, n_jobs=-1, random_state=46)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=46),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001413948B2D0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x0000014139489E90>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001413948A350>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000001413948AD10>},
                   random_state=46)

In [27]:
print(gs.best_params_)

{'max_depth': 27, 'min_impurity_decrease': 0.00013422643046617436, 'min_samples_leaf': 2, 'min_samples_split': 2}


In [28]:
np.max(gs.cv_results_['mean_test_score'])

0.8722353224254091

In [29]:
dt = gs.best_estimator_
dt.score(test_input, test_target)

0.8538461538461538